# Wildfire and call simulations

A simulation of a wildfire in the Paradise, CA area with emergency calls coming in. A map of the fire and the emergency calls is ploted using `folium`.

In [1]:
import datetime
import numpy as np
import pandas as pd

import folium
import folium.plugins
from process.folium_patch import HeatMapWithTimeAdditional

from process.simulation_calls import SimulationCalls
from process.simulation_fire import SimulationFire

from process.weather import Weather

In [2]:
# GEOGRAPHY
# Coordinates
map_center_coordinates = (39.759132, -121.619311) # Paradise # (40.750536, -122.548867)
map_radius = 1e-4

# Grid parameters
top_left = (39.903571, -121.867089) # Paradise 
bottom_right = (39.625599, -121.345045) # Paradise
grid_width = 0.001

# Epicenters for call simulation
random_epicenters = False
human_epicenters = {'Magalia': (39.777992, -121.639067),
                    'Pentz': (39.649350, -121.574862),
                    'Paradise': (39.758904, -121.627421)                   
                   }

background_epicenters = {'Paradise': (39.758904, -121.627421),                     
                        }

# Fire start
fire_starts = [(39.726949, -121.602515),
              ]

# Fire propagation
transfer_matrix = np.array([[0,1,0],[0,-0.4,1],[0,0,0]])#np.array([[0,1,0],[0,-0.8,1],[0,0,0]])

# Simulation parameters
n_steps = 50
simulation_time_delta = 10 #minutes

## Generating calls

In [3]:
calls_sim = SimulationCalls(map_center=map_center_coordinates,
                            map_radius=map_radius,
                            n_steps=n_steps,
                            call_centers=human_epicenters,
                            background_call_centers=background_epicenters,        
                           )
calls_df=calls_sim.run()

## Generating fire

In [4]:
fire_sim = SimulationFire(map_top_left=top_left,
                          map_bottom_right=bottom_right,
                          n_steps=n_steps,
                          grid_width=0.001,
                          transfer_matrix=transfer_matrix,
                          fire_starts=fire_starts,
                         )
fire_df=fire_sim.run()

In [5]:
fire_stations = pd.read_csv('./maps/paradise_firestations.csv', index_col=0)
fire_stations['UNITS_AVAILABLE'] = np.random.randint(low=2, high=5, size=len(fire_stations))
fire_stations['UNITS_DISPATCHED'] = np.random.randint(low=0, high=3, size=len(fire_stations))

## Generating map

In [6]:
def html_popup(name, address, available, dispatched):
    """
    Generate an html popup for folium.
    Input:
        name: str. Station name
        address: str. Station address
        available: str or int. Number of units available
        dispatched: str or int. Number of units dispatched
        
    Return:
        iframe: folium.IFrame
    """
    html=f"""
        <body style="background-color:#cecedd;">
        <font face="verdana" size=1>
        <h2>{name}</h2>
        <p>
        Address: {address}<br>
        Available units: {available}<br>
        Dispatched units: {dispatched}
        </p>
        </font>
        </body>
        """
    return folium.IFrame(html, width=300, height=100)



def fire_station_markers(dataframe):
    stations = folium.FeatureGroup('Fire Stations')

    for a in dataframe[['NAME', 'ADDRESS', 'Y', 'X', 'UNITS_AVAILABLE', 'UNITS_DISPATCHED']].values:
        station, address, lat, lon, available, dispatched = a
        stations.add_child(folium.features.RegularPolygonMarker(location=[lat,lon],
                                                                   popup=folium.Popup(html=html_popup(station, address, available, dispatched), max_width = 300),
                                                                   number_of_sides=4,
                                                                   radius=3,
                                                                  ))
    return stations

In [7]:
dtf = pd.read_pickle('maps/weather_20181101.df')
wea = Weather('maps/paradise_weather_grid.npy')
arrow = wea.wind_arrows(dtf)

In [8]:
def show_map(static=True):
    """
    Plot realtime fire map using folium.
    Input:
        static: bool. Default: True. Set to False to plot dynamic time map.
    Return:
        None
    """
    
    er_map = folium.Map(location=map_center_coordinates,
                          tiles='Stamen Terrain',#"CartoDB dark_matter",
                          width = 2*400,
                          height = 2*300,
                          zoom_start=12,
                         )
    
    # Marker
    er_map.add_child(fire_station_markers(fire_stations))
    
    # Wind direction
    er_map.add_child(arrow)
   
    # Static map
    if static:
        fire_heatmap = folium.plugins.HeatMap(fire_df[['latitude', 'longitude']].values, radius=10)
        fire_heatmap.layer_name = 'Fire'
        fire_heatmap.add_to(er_map)
        
        
        call_heatmap = folium.plugins.HeatMap(fire_df[['latitude', 'longitude']].values, radius=10)
        call_heatmap.layer_name = 'Calls'
        call_heatmap.add_to(er_map)
        

    # Dynamic map
    else:
        fire_data = [[[row['latitude'],row['longitude']] for _, row in fire_df[fire_df['time']==i].iterrows()] for i in range(n_steps)]
        start_date = datetime.datetime(2018, 11, 8, 17, 30)
        names = [(start_date+ i*datetime.timedelta(minutes=simulation_time_delta)).strftime('%D-%H-%M-%S') for i in range(n_steps)]
        
        fire_heatmap = folium.plugins.HeatMapWithTime(fire_data, 
                                                      radius = 10,
                                                      auto_play=True,
                                                      gradient={.4: '#FFA07A', .65: '#CD5C5C', 1: '#FF0000'},
                                                      index=names,
                                                     )
        fire_heatmap.add_to(er_map)
        fire_heatmap.layer_name = 'Fire'
        
        call_data = [[[row['latitude'],row['longitude']] for _, row in calls_df[calls_df['time'].isin(range(i-5,i+1))].iterrows()] for i in range(n_steps)]
        call_heatmap = HeatMapWithTimeAdditional(call_data, gradient={.4: '#FF7F00', .65: '#FF7F00', 1: '#FF7F00'}, radius=4)
        call_heatmap.layer_name = 'Calls'
        call_heatmap.add_to(er_map)
        
        
    folium.LayerControl().add_to(er_map)

    return er_map

show_map(static=False)    